# Find similar brand names with FuzzyWuzzy

There are different ways to make data dirty, and inconsistent data entry is one of them. Inconsistent values are even worse than duplicates, and sometimes difficult to detect.<br>
In this notebook, I apply [FuzzyWuzzy](https://github.com/seatgeek/fuzzywuzzy) package to find similar ramen brand names in a ramen review dataset.<br>
Data source: [The ramen rater Big list](https://www.theramenrater.com/resources-2/the-list/)

On my [GitHub](http://localhost:8888/notebooks/Desktop/Github/Python/Ramen/Find%20similar%20strings%20with%20FuzzyWuzzy.ipynb)

## Import

In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz

# Overview of the dataset

In [ ]:
# Load in the dataset
ramen = pd.read_csv('/kaggle/input/ramen-ratings-latest-update-jan-25-2020/Ramen_ratings_2020.csv')

# Display the first columns
ramen.head()

In [ ]:
ramen.drop('URL', axis=1, inplace=True)

In [ ]:
# Check data type and null values
ramen.info()

In [ ]:
# Remove leading and trailing spaces in each string value
for col in ramen[['Brand','Variety','Style','Country']]:
    ramen[col] = ramen[col].str.strip()
    print('Number of unique values in ' + str(col) +' is ' + str(ramen[col].nunique()))

In [ ]:
ramen['Country'].unique()

I would like to know if one brand can have multiple manufacturers in different countries.

In [ ]:
brand_country = ramen['Brand'] +' '+ ramen['Country']
brand_country.nunique()

586 is greater than 510 brands, so yes, one brand can have different country values. Next, we need to get the list of unique brand names.

In [ ]:
unique_brand = ramen['Brand'].unique().tolist()
sorted(unique_brand)[:20]

We can see some suspicious names right at the beginning. Let's have some tests first.

# FuzzyWuzzy

FuzzyWuzzy has four scorer options to find the Levenshtein distance between two strings. In this example, I would check on the token sort ratio and the token set ratio, for I believe they are more suitable for this dataset which might have mixed words order and duplicated words.<br>
I would pick four brand names and find their similar names in the Brand column. Since we're matching the Brand column with itself, the result would always include the selected name with a score of 100.

### Token Sort Ratio
The token sort ratio scorer tokenizes the strings and cleans them by returning these strings to lower cases, removing punctuations, and then sorting them alphabetically. After that, it finds the Levenshtein distance and returns the similarity percentage.

In [ ]:
process.extract('7 Select', unique_brand, scorer=fuzz.token_sort_ratio)

This result means that '7 Select/Nissin' has 70% similarity when referring to '7 Select'. Not bad if I set the threshold at 70% to get the pair of 7 Select - 7 Select/Nissin.

In [ ]:
process.extract('A-Sha', unique_brand, scorer=fuzz.token_sort_ratio)

Below 70%, there's no match for A-sha.

In [ ]:
process.extract('Acecook', unique_brand, scorer=fuzz.token_sort_ratio)

Still good at 70% threshold.

In [ ]:
process.extract("Chef Nic's Noodles", unique_brand, scorer=fuzz.token_sort_ratio)

Now, we have a problem here. Token sort ratio scorer will gets the wrong pair of Chef Nic's Noodles - Mr. Lee's Noodles if I set 70% threshold.

In [ ]:
process.extract('Chorip Dong', unique_brand, scorer=fuzz.token_sort_ratio)

This one looks good enough.

### Token Set Ratio
The token set ratio scorer also tokenizes the strings, and follows processing steps just like the token sort ratio. Then it collects common tokens between two strings and performs pairwise comparisons to find the similarity percentage.

In [ ]:
process.extract('7 Select', unique_brand, scorer=fuzz.token_set_ratio)

Since the token set ratio is more flexible, the score has increased from 70% to 100% for 7 Select - 7 Select/Nissin.

In [ ]:
process.extract('A-Sha', unique_brand, scorer=fuzz.token_set_ratio)

Now we see A-Sha has another name as A-Sha Dry Noodle. And we can see this only by using token set ratio.

In [ ]:
process.extract('Acecook', unique_brand, scorer=fuzz.token_set_ratio)

This one got 100% just like the 7 Select case.

In [ ]:
process.extract("Chef Nic's Noodles", unique_brand, scorer=fuzz.token_set_ratio)

This one got much worse when token set ratio returns 100% for the pair of Chef Nic's Noodles - S&S.

In [ ]:
process.extract('Chorip Dong', unique_brand, scorer=fuzz.token_set_ratio)

We have the same result for this one.

Although the token set ratio is more flexible and can detect more similar strings than the token sort ratio, it might also bring in more wrong matches.

# Apply FuzzyWuzzy in one column

### Token Sort Ratio

We need to create a dataframe with brand names, matched brands, and their scores.

In [ ]:
#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand, scorer=fuzz.token_sort_ratio)]

In [ ]:
#Create dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])
similarity_sort.head()

Since we're looking for matched values from the same column, one value pair would have another same pair in a reversed order. For example, we will find one pair of EDO Pack - Gau Do, and another pair of Gau Do - EDO Pack. To eliminate one of them later, I need to find a representative value for each of two same pairs.

In [ ]:
#Derive representative values
similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['brand_sort'], similarity_sort['match_sort'])
similarity_sort.head()

Based on the tests above, I would care about those pairs which have at least 80% similarity. I also exclude those which match to themselves (brand value and match value are exactly the same), and those which are duplicated pairs.

In [ ]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                                      (similarity_sort['brand_sort'] != similarity_sort['match_sort']) &
                                      (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'])]

In [ ]:
#Drop the representative value column
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()

Now, let's see the result.

In [ ]:
#Group matches by brand names and scores
#pd.set_option('display.max_rows', None)
high_score_sort.groupby(['brand_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

From the score of 95 and above, everything looks good. In each pair, the two values might have typos, one missing character, or inconsistent format, but overal they obviously refer to each other. Below 95, it would be harder to tell. We can look at some examples by listing out data from each pair.

In [ ]:
#Souper - Super - 91%
ramen[(ramen['Brand'] == 'Souper') | (ramen['Brand'] == 'Super')].sort_values(['Brand'])

For this pair, we see that these two brands come from different manufacturers, and there's also no similarity in their ramen types or styles. I would say that these brands are not the same.

In [ ]:
#Sura - Suraj - 89%
ramen[(ramen['Brand'] == 'Sura') | (ramen['Brand'] == 'Suraj')].sort_values(['Brand'])

Sura and Suraj are two different brands.

In [ ]:
#Ped Chef - Red Chef - 88%
ramen[(ramen['Brand'] == 'Ped Chef') | (ramen['Brand'] == 'Red Chef')].sort_values(['Brand'])

Here, we only have one record of the Ped Chef brand, and we also see the same pattern in its variety name in comparison with the Red Chef brand. I'm pretty sure these two brands are the same.

We can continue to check other pairs by the same method. From the threshold of 84% and below, we can ignore some pairs which are obviously different or we can make a quick check as above. Next, I will apply token set ratio scorer to find matched brand names, and then compare the results.

### Token Set Ratio

We will go over the same steps as above.

In [ ]:
#Create tuples of brand names, matched brand names, and the score
score_set = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand, scorer=fuzz.token_set_ratio)]

In [ ]:
#Create dataframe from the tuples and derive representative values
similarity_set = pd.DataFrame(score_set, columns=['brand_set','match_set','score_set'])
similarity_set['sorted_brand_set'] = np.minimum(similarity_set['brand_set'], similarity_set['match_set'])

#Pick values
high_score_set = similarity_set[(similarity_set['score_set'] >= 80) & 
                                    (similarity_set['brand_set'] != similarity_set['match_set']) & 
                                    (similarity_set['sorted_brand_set'] != similarity_set['match_set'])]

#Drop the representative value column
high_score_set = high_score_set.drop('sorted_brand_set',axis=1).copy()

Since the token set ratio scorer will tolerate more 'noise' when matching two values. I would group the result by matched values to reduce the number of rows.

In [ ]:
#Group brands by matches and scores
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
high_score_set.groupby(['match_set','score_set']).agg(
                       {'brand_set': ', '.join}).sort_values(
                       ['score_set'], ascending=False)

# Comparison

In this part, I will create a merged table including results from token sort ratio and token set ratio with some changes.<br>
The tables will be merged by matched values to shorten the result table, and because I would like to keep the scores after grouping all values, I need to create new columns which combine brand names and scores. 

In [ ]:
#Create columns with brand names combining scores
high_score_sort['brand_sort'] = high_score_sort['brand_sort'] + ': ' + high_score_sort['score_sort'].astype(str)
high_score_set['brand_set'] = high_score_set['brand_set'] + ': ' + high_score_set['score_set'].astype(str)

Now, I can group the two tables by matched names, and then rename the columns

In [ ]:
#Group data by matched name and store in new dataframe
token_sort = high_score_sort.groupby(['match_sort']).agg({'brand_sort': ', '.join}).reset_index()
token_set = high_score_set.groupby(['match_set']).agg({'brand_set': ', '.join}).reset_index()

#Rename columns
token_sort = token_sort.rename(columns={'match_sort':'brand'})
token_set = token_set.rename(columns={'match_set':'brand'})

I would choose 'outer' option to get all the data from two tables.

In [ ]:
#Outer join two tables by brand (matched names)
similarity = pd.merge(token_sort, token_set, how='outer', on='brand')

#Replace NaN values and rename columns for readability
similarity = similarity.replace(np.nan,'')
similarity = similarity.rename(columns={'brand_set':'token_set_ratio','brand_sort':'token_sort_ratio'})

In [ ]:
similarity.sort_values('brand')

Now we can see how different it is between two scorers. As expected, the token set ratio matches wrong names with high scores (e.g. S&S, Mr.Noodles). However, it does bring in more matches that the token sort ratio could not get (e.g. 7 Select/Nissin, Sugakiya Foods, Vina Acecook).<br>
It would be beneficial to apply both methods in this case.